# Exploring and clustering neighbourhoods in Toronto

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#conda install folium

!pip install folium
import folium # map rendering library

print('Libraries imported.') 

     |████████████████████████████████| 94 kB 5.1 MB/s  eta 0:00:01
Libraries imported.


In [3]:
#conda create --name myenv
#Activate the env
#conda activate myenv
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#### Creating Toronto data frame scraping from Wikipedia

In [4]:
import lxml.html as lh
#set url of origin
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#connect to the url
response = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(response.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [5]:
#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
    
for j in range(1,181):
    #T is our j'th row
    T=tr_elements[j]
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
Dict={title:column for (title,column) in col}
df = pd.DataFrame(Dict)
df = df.replace('\n','', regex=True)
df = df.rename(columns={'Postal Code\n':"Postal Code", 'Borough\n':"Borough", 'Neighbourhood\n':'Neighbourhood'})
df = df[~df.Borough.str.contains('Not assigned')]
df['Neighbourhood'] = df['Neighbourhood'].replace('Not assigned', df['Borough'])
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
#importing geolocation data
url2 = 'https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv'

dfgeolocation = pd.read_csv(url2)
dfgeolocation.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
#Adding geolocation to  the dataframe
df_merged = pd.merge(df, dfgeolocation, on='Postal Code')
df_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
#Filtering only Toronto
toronto_data = df_merged[df_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [10]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

En case of not being able to see the map, you may find a picture of it in the following link: https://github.com/verohy/Coursera_Capstone/blob/master/Toronto%20Neigbourhood%20map.png

#### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'KHLA0MRHSLCYLHZSLZAGPQX4QZL5NYKCFQSMRG2JAXUIWC2O' # your Foursquare ID
CLIENT_SECRET = 'LMFK1NJLDAGTPRSNPMHOOANCZ55WFZYWLISDH0NLUSAWBDDO' # your Foursquare Secret
ACCESS_TOKEN = 'DR4VOEHRB3NBWSSFJ0NEOYB43MNJ1SVDO1DQCXJ4C3J1C0JU'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
search_query = 'Marble_Hill'
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KHLA0MRHSLCYLHZSLZAGPQX4QZL5NYKCFQSMRG2JAXUIWC2O
CLIENT_SECRET:LMFK1NJLDAGTPRSNPMHOOANCZ55WFZYWLISDH0NLUSAWBDDO


#### Neighbourhood Clustering

In [17]:
#Getting rid of string columns
toronto_clustering = toronto_data[["Latitude","Longitude"]]
toronto_clustering.head()

,Latitude,Longitude
0,43.654260,-79.360636
1,43.662301,-79.389494
2,43.657162,-79.378937
3,43.651494,-79.375418
4,43.676357,-79.293031


In [18]:
# set number of clusters
kclusters = 5

#toronto_grouped_clustering = toronto_data.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 0, 0, 3, 0, 2], dtype=int32)

##### Through the k-mean method Toronto has been clustered in five areas

In [19]:
# add clustering labels
toronto_clustering.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_clustering with toronto_data to add latitude/longitude for each neighborhood

toronto_merged = pd.merge(toronto_clustering, toronto_merged, on=['Latitude',"Longitude"])

toronto_merged.head() # check the last columns!2

,Cluster Labels,Latitude,Longitude,Postal Code,Borough,Neighbourhood
0,0,43.654260,-79.360636,M5A,Downtown Toronto,"Regent Park, Harbourfront"
1,0,43.662301,-79.389494,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
2,0,43.657162,-79.378937,M5B,Downtown Toronto,"Garden District, Ryerson"
3,0,43.651494,-79.375418,M5C,Downtown Toronto,St. James Town
4,1,43.676357,-79.293031,M4E,East Toronto,The Beaches


#### Visualizing the resulting clusters

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

En case of not being able to see the map above, you may find a picture of it in the following link: https://github.com/verohy/Coursera_Capstone/blob/master/Toronto%20Clustering.png